## Sample Script for 1 df


- hz for each to_csv#
- anfangen mit preprocessing :)

# TODO
- find out how red marked stimuli section in imotions are saved in the data

In [1]:
# TODO: Round all timestamps to remove floating point weirdness
# 447,929.23030000000006, ...
# 448,929.2894, ...
# -> add as function to transformations of raw data, even imotions data?

In [2]:
import os
import sys
from pathlib import Path


PROJECT_DIR = Path.cwd()
if PROJECT_DIR.stem == 'data':
    PROJECT_DIR = PROJECT_DIR.parents[1]
    sys.path.insert(0, PROJECT_DIR.as_posix())
    os.chdir(PROJECT_DIR.as_posix())
    %load_ext autoreload
    %autoreload 2

In [3]:
from typing import List, Union, Optional, Tuple, Dict, Any

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from src.helper_func import flatten_list, print_size
from src.data.config_participant import PARTICIPANT_LIST, ParticipantConfig
from src.data.config_data import DataConfigBase
from src.data.config_data_imotions import IMOTIONS_LIST, iMotionsConfig
from src.data.config_data_raw import RAW_LIST, RawConfig
from src.data.config_data_trial import TRIAL_LIST, TrialConfig
from src.data.process_data import load_participant_datasets, load_participant_datasets_pl, Participant, Data, save_participant_datasets
from src.data.transform_data import apply_func_participant, create_trial_index, reorder_multiindex, interpolate, resample, resample_to_500hz, standardize, normalize

pd.options.mode.chained_assignment = 'warn'  # default='warn'


In [4]:
# participant_raw = load_participant_datasets_pl(PARTICIPANT_LIST[0], RAW_LIST)
participant = load_participant_datasets(PARTICIPANT_LIST[0], TRIAL_LIST)

In [17]:
participant.eeg.index.names

FrozenList([None])

In [ ]:
def interpolate(df, method='linear'):
    columns_to_interpolate = df.columns[(df.dtypes == float)]
    if 'Trial' in df.index.names:
        df[columns_to_interpolate] = df.groupby('Trial')[columns_to_interpolate].transform(lambda x: x.interpolate(method=method))
    else:
        df[columns_to_interpolate] = df[columns_to_interpolate].interpolate(method=method)
    return df

a = interpolate(participant.rating)
# count nan values
a.isna().sum()

Time         0
Timestamp    0
Rating       0
dtype: int64

In [ ]:


def create_trial_index(df):
    """Create a trial index based on the stimuli seed which is originally send once at the start and end of each trial."""
    # Check if the trial index already exists
    if 'Trial' in df.index.names:
        return df
    # 0. Check if all trials are complete  # TODO
    # 1. Forward fill and backward fill columns
    ffill = df['Stimuli_Seed'].ffill()
    bfill = df['Stimuli_Seed'].bfill()
    # 2. Where forward fill and backward fill are equal, replace the NaNs in the original Stimuli_Seed
    df['Stimuli_Seed'] = np.where(ffill == bfill, ffill, df['Stimuli_Seed'])
    # 3. Only keep rows where the Stimuli_Seed is not NaN
    df = df[df['Stimuli_Seed'].notna()]
    df['Stimuli_Seed'] = df['Stimuli_Seed'].astype(int)
    # Create a new column that contains the trial number
    df['Trial'] = df.Stimuli_Seed.diff().ne(0).cumsum()
    # Add Trial to the index
    df.set_index('Trial', append=True if 'Time' in df.index.names else False, inplace=True)
    return df

participant = add_stimuli_seed_column_participant(participant)
display(participant.eda)
participant_ = apply_func_participant(participant, create_trial_index)
display(participant_.temperature)

,Timestamp,EDA_d_Battery,EDA_RAW,EDA_d_PacketReceptionRate,Stimuli_Seed
Trial,,,,,
1,1.880411e+05,NaN,NaN,NaN,1
1,1.880461e+05,3638.112821,6.057357,100.0,1
1,1.880491e+05,3659.958974,6.062463,100.0,1
1,1.880491e+05,3655.589744,6.057357,100.0,1
1,1.880640e+05,3620.635897,6.057357,100.0,1
...,...,...,...,...,...
6,2.303530e+06,3617.723077,4.520683,99.0,6
6,2.303537e+06,3630.830769,4.515577,99.0,6
6,2.303552e+06,3617.723077,4.520683,99.0,6


,Timestamp,Temperature,Stimuli_Seed
Trial,,,
1,1.880411e+05,NaN,1
1,1.880471e+05,38.600,1
1,1.880820e+05,38.600,1
1,1.880979e+05,38.600,1
1,1.881139e+05,38.600,1
...,...,...,...
6,2.303465e+06,40.982,6
6,2.303482e+06,40.983,6
6,2.303499e+06,40.983,6


In [ ]:
def apply_func_participant(participant, func):
    #TODO: use map instead, e.g.:
    # dict(zip(a, map(f, a.values())))
    # dict(map(lambda item: (item[0], f(item[1])), my_dictionary.items()
    for data in participant.datasets:
        participant.datasets[data].dataset = func(participant.datasets[data].dataset)
    return participant


In [ ]:
def create_timedelta_index(df):
    """Convert the time stamp to time delta and set it as index."""
    # just casting to timedelta64[ms] is faster but less accurate
    df["Time"] = pd.to_timedelta(df["Timestamp"], unit='ms').round('ms').astype('timedelta64[ms]')
    df.set_index("Time", append=True if 'Trial' in df.index.names else False, inplace=True)
    # Remove duplicate index
    df = df[~df.index.duplicated(keep='first')]
    return df

def create_timedelta_index_participant(participant):
    participant.datasets = {k: create_timedelta_index(v) for k, v in participant.datasets.items()}
    return participant


participant = apply_func_participant(participant, create_timedelta_index)
participant

Participant(id='001_pilot_bjoern', datasets={'trial': Data(name='trial', dataset=                           Timestamp  Stimuli_Seed
Time                                              
0 days 00:03:08.041000  1.880411e+05             1
0 days 00:07:48.086000  4.680859e+05             1
0 days 00:09:09.707000  5.497068e+05             2
0 days 00:13:49.753000  8.297534e+05             2
0 days 00:14:59.310000  8.993097e+05             3
0 days 00:19:39.322000  1.179322e+06             3
0 days 00:21:36.641000  1.296641e+06             4
0 days 00:26:16.673000  1.576673e+06             4
0 days 00:28:19.855000  1.699855e+06             5
0 days 00:32:59.885000  1.979885e+06             5
0 days 00:33:43.541000  2.023541e+06             6
0 days 00:38:23.572000  2.303572e+06             6), 'temperature': Data(name='temperature', dataset=                           Timestamp  Temperature
Time                                             
0 days 00:03:08.047000  1.880471e+05       38.600
0 day

In [ ]:
def get_sampling_rate_of_df(df):
    start = pd.to_timedelta(df['Timestamp'].iloc[0])
    end = pd.to_timedelta(df['Timestamp'].iloc[-1])
    duration = end - start
    duration_in_s = duration.total_seconds()
    sampling_rate = len(df) / duration_in_s
    return sampling_rate


get_sampling_rate_of_df(df)

NameError: name 'df' is not defined

In [ ]:
# Write code for quality check based on raw data
# - packet reception rate
# - battery level
# - constant sample rate over each trial

In [ ]:
# Example usage
base_path = 'data/external/'
participant_ids =  [
    '001_pilot_bjoern',
    #'002_pilot_melis'
    ]
from src.data.info_external import EXTERNAL_LIST

data_infos = [
    TRIAL,
    RATING,
    TEMPERATURE,
    EDA,
    ECG,
    EEG,
    PUPILLOMETRY,
    AFFECTIVA,
    SYSTEM
]


participants = [load_participant_data(participant_id, base_path, data_infos) for participant_id in participant_ids]
participants_df = {participant.id: participant.to_dataframe() for participant in participants}

sampling_rate = 128

df = participants_df[list(participants_df)[0]]
#df = create_timedelta_index(df)
#df = create_trial_index(df)
# df = reorder_multiindex(df)
# df = resample_to_500hz(df)
# df = resample(df, ms=1000/sampling_rate)
# df = interpolate(df)
# df = normalize(df)
#df = standardize(df)
print('df size:', round(sys.getsizeof(df) / 1024**2), 'MB')
df.isna().sum()

ModuleNotFoundError: No module named 'src.data.info_external'

In [ ]:
EXTERNAL_DICT.values()

dict_values([ExternalInfo(name='trial', path='ExternalMarker_ET_EventAPI_ExternDevice.csv', imotions_columns=['RowNumber', 'Timestamp', 'MarkerName', 'MarkerDescription', 'MarkerType', 'SceneType'], keep_columns=['Timestamp', 'MarkerDescription'], rename_columns=None, device_sampling_rate=None, notes='MarkerDescription will be renamed to Stimuli_Seed,         it originally contains the stimulus seed and is send once at the start and end of each trial.'), ExternalInfo(name='temperature', path='TemperatureCurve_TemperatureCurve@1_ET_EventAPI_ExternDevice.csv', imotions_columns=['RowNumber', 'Timestamp', 'Temperature'], keep_columns=['Timestamp', 'Temperature'], rename_columns=None, device_sampling_rate=None, notes=None), ExternalInfo(name='rating', path='RatingCurve_RatingCurve@1_ET_EventAPI_ExternDevice.csv', imotions_columns=['RowNumber', 'Timestamp', 'Rating'], keep_columns=['Timestamp', 'Rating'], rename_columns=None, device_sampling_rate=None, notes=None), ExternalInfo(name='eda', p

In [ ]:
# Example usage
base_path = 'data/raw/'
participant_ids =  [
    '001_pilot_bjoern',
    #'002_pilot_melis'
    ]

data_infos = [
    #TRIAL,
    #RATING,
    #TEMPERATURE,
    #EDA,
    #ECG,
    EEG,
    #PUPILLOMETRY,
    #AFFECTIVA,
    #SYSTEM
]

participants = [load_participant_data(participant_id, base_path, data_infos) for participant_id in participant_ids]
participants_df = {participant.id: participant.to_dataframe() for participant in participants}

sampling_rate = 128

df = participants_df[list(participants_df)[0]]
#df = create_timedelta_index(df)
#df = create_trial_index(df)
# df = reorder_multiindex(df)
# df = resample_to_500hz(df)
# df = resample(df, ms=1000/sampling_rate)
# df = interpolate(df)
# df = normalize(df)
#df = standardize(df)
print('df size:', round(sys.getsizeof(df) / 1024**2), 'MB')
df.isna().sum()

TypeError: reduce() of empty iterable with no initial value

In [ ]:
# Count nan values
df.isna().sum()

Timestamp                     0
Stimuli_Seed                  0
EDA_d_Battery                12
EDA_RAW                      12
EDA_d_PacketReceptionRate    12
dtype: int64

In [ ]:
import pandas as pd

def calculate_sample_rate(df, trial, duration_minutes):
    minute = pd.Timedelta(duration_minutes, unit='m')
    start = df.loc[trial].iloc[0].name
    end = start + minute
    num_samples = len(df.loc[trial].loc[start:end]['GSR Conductance CAL'].dropna())
    sample_rate = num_samples / minute.total_seconds()
    return sample_rate


In [ ]:
    start = df.loc[trial].index.min()
    end = df.loc[trial].index.max()
    duration = end - start
    duration_seconds = duration.total_seconds()
    num_samples = len(df.loc[trial]['GSR Conductance CAL'].dropna())
    sample_rate = num_samples / duration_seconds

NameError: name 'trial' is not defined

In [ ]:
df.groupby('Trial')


In [ ]:
df.loc[1].iloc[-1].name - df.loc[1].iloc[0].name 

Timedelta('0 days 00:04:40.045000')

In [ ]:
trial = 1
minute = pd.Timedelta(4.66666666666666, unit='m')

start = df.loc[trial].iloc[0].name
end = start + minute
num_samples = len(df.loc[trial].loc[start:end]['GSR Conductance CAL'].dropna())

sample_rate = num_samples / minute.total_seconds()
sample_rate, minute.total_seconds(), num_samples

(100.05, 280.0, 28014)

In [ ]:
start = df.loc[trial].iloc[0].Timestamp
end = df.loc[trial].iloc[-1].Timestamp
time = (end - start)
time = time # / 1000 / 60
time = pd.Timedelta(time, unit='ms')
num_samples / time.total_seconds(), time.total_seconds(), num_samples

(100.03399277483473, 280.044805, 28014)

In [ ]:
280 / 60

4.666666666666667

### Sanity checks
TODO: make unit test

In [ ]:
# Check stimuli length, must be >= stimuli.duration
# where we create a stimulus object from the config file with the right seed, etc.
# Also check after resampling, interpolation, etc.
(df.groupby('Trial').last().Timestamp - df.groupby('Trial').first().Timestamp).astype('timedelta64[ms]')


Trial
1   0 days 00:04:40.044000
2   0 days 00:04:40.046000
3   0 days 00:04:40.011000
4   0 days 00:04:40.031000
5   0 days 00:04:40.029000
6   0 days 00:04:40.030000
Name: Timestamp, dtype: timedelta64[ms]

In [ ]:
(df.groupby('Trial').last())

,Timestamp,MarkerDescription,System Timestamp CAL,GSR Conductance CAL,Packet reception rate RAW
Trial,,,,,
1,4.680859e+05,1,1.702981e+12,8.870340,100.0
2,8.297534e+05,2,1.702981e+12,8.752920,99.0
3,1.179322e+06,3,1.702982e+12,7.139666,99.0
4,1.576673e+06,4,1.702982e+12,9.421705,99.0
5,1.979885e+06,5,1.702982e+12,5.332414,99.0
6,2.303572e+06,6,1.702983e+12,4.520683,99.0


In [ ]:
# Control data quality


In [ ]:
# search for nan values in raw data (trial + data)

In [ ]:
import os
import sys
from pathlib import Path


PROJECT_DIR = Path.cwd()
if PROJECT_DIR.stem == 'data':
    PROJECT_DIR = PROJECT_DIR.parents[1]
    sys.path.insert(0, PROJECT_DIR.as_posix())
    os.chdir(PROJECT_DIR.as_posix())
    %load_ext autoreload
    %autoreload 2

In [ ]:
import os
from pathlib import Path
from functools import reduce
from dataclasses import dataclass
from typing import Dict, List
import logging

import pandas as pd
import polars as pl

from src.helper_func import flatten_list, print_size
from src.data.config_participant import PARTICIPANT_LIST, ParticipantConfig
from src.data.config_data import DataConfigBase
from src.data.config_data_imotions import IMOTIONS_LIST, iMotionsConfig
from src.data.config_data_raw import RAW_LIST, RawConfig
from src.data.config_data_trial import TRIAL_LIST, TrialConfig
from src.data.process_data import load_participant_datasets, Participant, Data
from src.data.transform_data import create_trial_index, reorder_multiindex, interpolate, resample, resample_to_500hz, standardize, normalize

# use polars for faster loading of pd.DataFrame
# note that it seems that the time stamp sometimes 
import polars as pl


def load_dataset_pl(
        participant_config: ParticipantConfig,
        data_config: DataConfigBase
        ) -> Data:

    data_path = data_config.load_dir / participant_config.id / f"{participant_config.id}_{data_config.name}.csv"
    data_start_index = 0

    if isinstance(data_config, iMotionsConfig):
        data_path = data_config.load_dir / participant_config.id / f"{data_config.name_imotions}.csv"
        with open(data_path, 'r') as file:
            lines = file.readlines(2**16)
        data_start_index = next(i for i, line in enumerate(lines) if "#DATA" in line) + 1

    # Load and process data using Polars
    dataset = pl.read_csv(
        data_path, 
        columns=data_config.load_columns,
        skip_rows=data_start_index,
    ).to_pandas()
    
    if isinstance(data_config, iMotionsConfig):
        if data_config.rename_columns:
            dataset = dataset.rename(data_config.rename_columns)

    return Data(name=data_config.name, dataset=dataset)

def load_participant_datasets_pl(
        participant_config: ParticipantConfig, 
        data_configs: List[DataConfigBase]
        ) -> Participant:

    datasets: Dict[str, Data] = {}
    for data_config in data_configs:
        datasets[data_config.name] = load_dataset_pl(participant_config, data_config)
        
    return Participant(id=participant_config.id, datasets=datasets)


In [ ]:
a = load_participant_datasets_pl(PARTICIPANT_LIST[0], RAW_LIST)
a.eeg.dtypes

Timestamp      float64
EEG_RAW_Ch1    float64
EEG_RAW_Ch2    float64
EEG_RAW_Ch3    float64
EEG_RAW_Ch4    float64
EEG_RAW_Ch5    float64
EEG_RAW_Ch6    float64
EEG_RAW_Ch7    float64
EEG_RAW_Ch8    float64
dtype: object

In [ ]:
from typing import List, Union, Optional, Tuple, Dict, Any

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from src.helper_func import flatten_list, print_size
from src.data.config_participant import PARTICIPANT_LIST, ParticipantConfig
from src.data.config_data import DataConfigBase
from src.data.config_data_imotions import IMOTIONS_LIST, iMotionsConfig
from src.data.config_data_raw import RAW_LIST, RawConfig
from src.data.config_data_trial import TRIAL_LIST, TrialConfig
from src.data.process_data import load_participant_datasets, Participant, Data
from src.data.transform_data import create_trial_index, reorder_multiindex, interpolate, resample, resample_to_500hz, standardize, normalize

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
participant_raw = load_participant_datasets(PARTICIPANT_LIST[0], RAW_LIST)
# participant_trial = load_participant_datasets(PARTICIPANT_LIST[0], TRIAL_LIST)
participant_raw.eeg.dtypes

Timestamp      float64
EEG_RAW_Ch1    float64
EEG_RAW_Ch2    float64
EEG_RAW_Ch3    float64
EEG_RAW_Ch4    float64
EEG_RAW_Ch5    float64
EEG_RAW_Ch6    float64
EEG_RAW_Ch7    float64
EEG_RAW_Ch8    float64
dtype: object

In [ ]:
a.eeg.equals(participant_raw.eeg)

False

In [ ]:
a.eeg.iloc[444].Timestamp

928.5974000000001

In [ ]:
participant_raw.eeg.iloc[444].Timestamp

928.5974

In [ ]:
pd.concat([a.eeg,participant_raw.eeg]).drop_duplicates(keep=False)

,Timestamp,EEG_RAW_Ch1,EEG_RAW_Ch2,EEG_RAW_Ch3,EEG_RAW_Ch4,EEG_RAW_Ch5,EEG_RAW_Ch6,EEG_RAW_Ch7,EEG_RAW_Ch8
444,928.5974,-4360.150879,-6395.196777,12322.378906,-2388.048096,1926.135986,6642.961914,-3906.773926,-2549.267090
458,963.2437,-4414.368164,-6378.507812,12324.239258,-2397.250977,1921.701050,6636.429199,-3905.677979,-2556.133057
481,999.1812,-4365.826172,-6343.937012,12323.190430,-2392.815918,1927.566040,6630.611816,-3913.259033,-2553.033936
45007,90077.0919,-4199.981934,-6412.315918,12451.316406,-2393.341064,2019.453003,6674.099121,-3951.835938,-2479.839111
45016,90077.5346,-4181.670898,-6386.375977,12448.073242,-2432.966064,2029.609009,6662.369141,-3972.101074,-2481.031006
...,...,...,...,...,...,...,...,...,...
499880,999840.6934,-4140.710938,-6210.565918,14117.910156,-2200.698975,2576.541992,6751.728027,-4021.311035,-2337.599121
499934,999948.4741,-4152.488770,-6223.059082,14102.173828,-2212.666992,2559.662109,6723.880859,-4040.812988,-2363.729004
499947,999983.6076,-4146.003906,-6222.058105,14109.421875,-2199.315918,2570.152100,6745.577148,-4021.406006,-2346.802002
499957,999984.9778,-4154.873047,-6228.399902,14095.498047,-2209.663086,2560.520020,6749.105957,-4025.745117,-2356.576904
